# Initialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim, length

# import trimming function
from script.silver.silver_utils import trim_string_columns

# Read Bronze table

In [0]:
df = spark.table("workspace.bronze.crm_sales_details")

# Silver Transformations

## Trimming

In [0]:
df = trim_string_columns(df)

## Cleaning Dates

In [0]:
df = (
    df.
    withColumn(
        "sls_order_dt",
        F.when(
            (col("sls_order_dt") == 0) | (length(col("sls_order_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_order_dt").cast("string"),
        "yyyMMdd"))
    )
    .withColumn(
        "sls_ship_dt",
        F.when(
            (col("sls_ship_dt") == 0) | (length(col("sls_ship_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_ship_dt").cast("string"),
        "yyyMMdd"))
    )
    .withColumn(
        "sls_due_dt",
        F.when(
            (col("sls_due_dt") == 0) | (length(col("sls_due_dt")) != 8),
            None
        ).otherwise(F.to_date(col("sls_due_dt").cast("string"),
        "yyyMMdd"))
    )
)

## Sales and Price Corrections

In [0]:
df = (
    df
    .withColumn(
        "sls_price",
        F.when(
            (col("sls_price").isNull()) | (col("sls_price") <= 0),
           F.when(
               col("sls_quantity") != 0,
               col("sls_sales") / col("sls_quantity")
           ).otherwise(None)
        ).otherwise(col("sls_price"))
    )
)

In [0]:
rename_map = {
    "sls_ord_num": "order_number",
    "sls_prd_key": "product_number",
    "sls_cust_id": "customer_id",
    "sls_order_dt": "order_date",
    "sls_ship_dt": "ship_date",
    "sls_due_dt": "due_date",
    "sls_sales": "sales_amount",
    "sls_quantity": "quantity",
    "sls_price": "price"
}

for old_name, new_name in rename_map.items():
    df = df.withColumnRenamed(old_name, new_name)

## Sanity checks of dataframe

In [0]:
df.limit(10).display()

In [0]:
# should not return price = 0 or price < 0
display(
    df.groupBy("price")
      .agg(F.count("*").alias("row_count"))
      .orderBy("price")
)

# Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_sales")

## Sanity checks for silver table

In [0]:
%sql
SELECT *
FROM workspace.silver.crm_sales
LIMIT 10;